In [0]:
display(dbutils.fs.ls('/Volumes/workspace/alura/c1/wines/'))

In [0]:
file_location = '/Volumes/workspace/alura/c1/wines/winequality-red.csv'
file_type = 'csv'
infer_schema = 'true'
first_row_is_header = 'true'
delimiter = ';'

df = spark.read.csv(file_location, sep = delimiter, header=True, inferSchema=True)

for c in df.columns:
    new_col = c.strip().lower().replace(" ", "_")
    df = df.withColumnRenamed(c, new_col)

table_name = "workspace.alura.redwine"

df.write.format("delta").mode("overwrite").saveAsTable(table_name)

In [0]:
file_location = '/Volumes/workspace/alura/c1/wines/winequality-white.csv'
file_type = 'csv'
infer_schema = 'true'
first_row_is_header = 'true'
delimiter = ';'

df = spark.read.csv(file_location, sep = delimiter, header=True, inferSchema=True)

for c in df.columns:
    new_col = c.strip().lower().replace(" ", "_")
    df = df.withColumnRenamed(c, new_col)

table_name = "workspace.alura.whitewine"

df.write.format("delta").mode("overwrite").saveAsTable(table_name)

In [0]:
%sql

select * from workspace.alura.redwine limit 10

In [0]:
%sql
select * from workspace.alura.whitewine limit 10

In [0]:
%sql

USE alura

In [0]:
%sql 

DESCRIBE whitewine

In [0]:
%sql
SELECT DISTINCT (quality)
  FROM redwine
    ORDER BY quality DESC

In [0]:
%sql
SELECT quality, MIN (pH) AS pH_mimino, MAX (pH) AS pH_maximo    
  FROM redwine
    GROUP BY quality
      ORDER BY quality DESC


In [0]:
%sql
CREATE OR REPLACE TABLE new_red_wine 
  AS SELECT *, 'red' AS wine_type
    FROM redwine


In [0]:
%sql
SELECT *
  FROM new_red_wine


In [0]:
%sql
CREATE OR REPLACE TABLE new_white_wine 
  AS select *, 'white' AS wine_type
    FROM whitewine


In [0]:
%sql
SELECT *
  FROM new_white_wine


In [0]:
%sql
CREATE OR REPLACE TABLE combined_wines 
  AS SELECT *
    FROM new_red_wine
      UNION ALL SELECT *
        FROM new_white_wine;


In [0]:
%sql
SELECT wine_type, AVG (pH) AS pH_medio    
  FROM combined_wines
    GROUP BY wine_type
      ORDER BY wine_type


In [0]:
%sql

SHOW TABLES

In [0]:
tabela = spark.table('alura.combined_wines')

In [0]:
display(tabela)

In [0]:
spark.sql('''
          SELECT DISTINCT (quality)
          FROM combined_wines
          ORDER BY quality DESC
          ''').show()

In [0]:
resultado = spark.sql('''
    SELECT *
        FROM combined_wines
            WHERE pH < 3
''')

In [0]:
type(resultado)

In [0]:
resultado.createOrReplaceTempView('temp_ph_lower_than_3')

In [0]:
spark.sql('''
    SELECT quality, COUNT (quality) AS Freq
        FROM temp_ph_lower_than_3
            GROUP BY quality
''').show()